In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf

tf.keras.backend.clear_session()

2024-07-31 11:13:01.649430: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-31 11:13:01.780078: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 11:13:01.780142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 11:13:01.799664: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-31 11:13:01.840809: I tensorflow/core/platform/cpu_feature_guar

In [3]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime

from tqdm import tqdm

In [4]:
# GPU settings



gpus = tf.config.list_physical_devices('GPU')



# Setting gpu[0] as main device
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)


# Setting memory growth
try:
  tf.config.experimental.set_memory_growth(gpus[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass



# tf.debugging.set_log_device_placement(True)


os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(os.getenv('TF_GPU_ALLOCATOR'))

1 Physical GPUs, 1 Logical GPU


2024-07-31 11:13:03.505455: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 11:13:03.574284: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-31 11:13:03.574584: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

from sklearn.preprocessing import OneHotEncoder

In [7]:
class VGGNet(Sequential):
    def __init__(self, input_shape, num_classes):
        super().__init__()
        
        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu",
            input_shape=input_shape
        ))

        self.add(Conv2D(
            64,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            128,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            256,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(Conv2D(
            512,
            kernel_size=(3,3),
            kernel_initializer="he_normal",
            padding="same",
            activation="relu"
        ))

        self.add(MaxPooling2D(
            pool_size=(2, 2),
            strides=2
        ))

        self.add(Flatten()),

        self.add(Dense(4096, activation="relu"))
        self.add(Dense(4096, activation="relu"))
        self.add(Dense(1000, activation="softmax"))

        self.compile(
            optimizer=Adam(0.001),
            loss="categorical_crossentropy",
            metrics=["accuracy"]
        )

In [8]:
num_epochs = 1
batch_size = 2
img_height = 224
img_width = 224
Input_shape = (img_width, img_height, 3)
num_classes = 3


model = VGGNet(Input_shape, num_classes=num_classes)

model.summary()

Model: "vgg_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                      

In [9]:
train_images_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/synsets"
valid_images_dir = "/home/abhinav/Documents/Work/2 Hobby_projects/Datasets/Imagenet/valid_synsets"


train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

train_generator = train_datagen.flow_from_directory(
    train_images_dir,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=1,
    class_mode="categorical"
)

valid_datagen = ImageDataGenerator(
    rescale=1./255
)

valid_generator = valid_datagen.flow_from_directory(
    valid_images_dir,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    shuffle=True,
    seed=1,
    class_mode="categorical"
)

train_images_num = train_generator.samples
valid_images_num = valid_generator.samples

Found 3605 images belonging to 3 classes.
Found 264 images belonging to 3 classes.


In [10]:
log_dir = "./logs/fit" + datetime.datetime.now().strftime("%d%M%Y-%H%M%S")
tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
callbacks_list = [tensorboard_callbacks]

In [11]:
# Since lack of gpu power, we cant train this model in local machine, so we take step to take pretrained model

# model.fit(
#     train_generator,
#     epochs=num_epochs,
#     steps_per_epoch=train_images_num//batch_size,
#     validation_data=valid_generator,
#     validation_steps=valid_images_num//batch_size,
#     callbacks=callbacks_list,
#     verbose=1
# )

In [12]:
from keras.applications.vgg16 import VGG16
vggmodel = VGG16(weights='imagenet', include_top=True)

2024-07-31 11:13:14.204750: W external/local_tsl/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 392.00MiB (rounded to 411041792)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-07-31 11:13:14.204806: I external/local_tsl/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-07-31 11:13:14.204821: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 40, Chunks in use: 40. 10.0KiB allocated for chunks. 10.0KiB in use in bin. 1.2KiB client-requested in use in bin.
2024-07-31 11:13:14.204830: I external/local_tsl/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 4, Chunks in use: 4. 2.0KiB allocated for chunks. 2.0KiB in use in bin. 2.0KiB client-requested in use in bin.
2024-07-31 11:13:14.204837: 

ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 

In [ ]:
print("layers of the model:")
for la in vggmodel.layers:
    print(la)


print("\n\nlayers that we are going to set trainable to False")
for layers in (vggmodel.layers)[:15]:
    print(layers)
    layers.trainable = False

X= vggmodel.layers[-2].output
predictions = Dense(2, activation="softmax")(X)
model_final = Model(vggmodel.input, predictions)
opt = Adam(learning_rate=0.0001)
model_final.compile(loss = keras.losses.categorical_crossentropy, optimizer = opt, metrics=["accuracy"])
model_final.summary()